In [1]:
import numpy as np
import pandas as pd
X = np.array([[1,'S'],[1,'M'],[1,'M'],[1,'S'],[1,'S'],\
              [2,'S'],[2,'M'],[2,'M'],[2,'L'],[2,'L'],\
              [3,'L'],[3,'M'],[3,'M'],[3,'L'],[3,'L']])
Y = np.array([-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1]).reshape(-1,1)

In [2]:
class NaiveBayes:
    def __init__(self,lam=1):
        self.lam = lam
    
    def fit(self,X,Y):
        data = pd.concat([pd.DataFrame(X,columns=[str(i) for i in range(X.shape[1])]),\
        pd.DataFrame(Y,columns=['Y'])],axis=1)
        ## 计算先验概率
        self.piror = dict(data['Y'].value_counts())
        self.K = len(self.piror)
        ## 计算条件概率
        self.S = {}
        self.CondProb = {}
        for col in data.columns[:-1]:
            tmp = data[[col,'Y']]
            tmp = tmp.groupby([col,'Y']).agg({'Y':'count'})
            tmp.columns=['count']
            tmp = tmp.reset_index()
            self.CondProb[col] = tmp
            self.S[col] = len(tmp[col].value_counts())
            
    def predict(self,new_X):
        posterior = []
        for k in self.piror.keys():
            piror = (self.piror[k]+self.lam)/(sum(self.piror.values())+self.lam*self.K)
            for i in range(new_X.shape[1]):
                S = self.S[str(i)]
                cond = self.CondProb[str(i)]
                count = cond.loc[(cond[str(i)]==new_X[0,i])&(cond['Y']==k),['count']]
                count = (count.values[0,0]+self.lam)/(self.piror[k]+S*self.lam)
                piror *= count
            posterior.append(piror)
        posterior = [round(p/sum(posterior),2) for p in posterior]
        return dict(zip(self.piror.keys(),posterior))

# Generate and Predict

In [3]:
clf = NaiveBayes(lam=0)
clf.fit(X,Y)
clf.predict(np.array([[2,'S']]))

{1: 0.25, -1: 0.75}